In [2]:
import numpy as np
import pandas as pd
import pickle
import sys
sys.path.append('../src')
from copula_simulation import fit_copula_and_save, load_copula_and_marginals, simulate_copula_portfolio, comprehensive_analysis

In [ ]:
def main():
    parser = argparse.ArgumentParser(description="Copula-based portfolio risk simulation and fitting.")
    subparsers = parser.add_subparsers(dest='command')

    # Subparser for simulation
    sim_parser = subparsers.add_parser('simulate', help='Simulate portfolio risk using fitted copula')
    sim_parser.add_argument('--copula', type=str, default='models/best_copula.pkl', help='Path to copula model pickle file')
    sim_parser.add_argument('--std_resids', type=str, default='data/processed/std_resids.csv', help='Path to standardized residuals CSV')
    sim_parser.add_argument('--n_sim', type=int, default=10000, help='Number of simulations')
    sim_parser.add_argument('--output', type=str, default='data/processed/simulated_copula_samples.csv', help='Output CSV file for simulated samples')

    # Subparser for fitting copula
    fit_parser = subparsers.add_parser('fit', help='Fit copula model and save to pickle')
    fit_parser.add_argument('--std_resids', type=str, default='data/processed/std_resids.csv', help='Path to standardized residuals CSV')
    fit_parser.add_argument('--copula_type', type=str, default='vine_student', choices=['vine_student','vine_gaussian','gaussian','clayton','gumbel'], help='Type of copula to fit (vine_student, vine_gaussian, gaussian, clayton, gumbel)')
    fit_parser.add_argument('--output', type=str, default='models/best_copula.pkl', help='Output path for copula pickle')

    # Subparser for analysis
    analyze_parser = subparsers.add_parser('analyze', help='Comprehensive analysis of simulated results')
    analyze_parser.add_argument('--sim_csv', type=str, default='data/processed/simulated_copula_samples.csv', help='Path to simulated samples CSV')
    analyze_parser.add_argument('--real_csv', type=str, default=None, help='Path to real data CSV (optional)')
    analyze_parser.add_argument('--alpha', type=float, default=0.05, help='VaR/ES quantile (default 0.05)')
    analyze_parser.add_argument('--out_dir', type=str, default='reports/figures', help='Output directory for figures')

    args = parser.parse_args()

    if args.command == 'fit':
        fit_copula_and_save(args.std_resids, args.copula_type, args.output)
    elif args.command == 'simulate':
        print(f"Loading copula model from: {args.copula}")
        print(f"Loading standardized residuals from: {args.std_resids}")
        best_copula_model, marginals = load_copula_and_marginals(args.copula, args.std_resids)
        print(f"Simulating {args.n_sim} samples...")
        sim_data = simulate_copula_portfolio(best_copula_model, marginals, n_sim=args.n_sim)
        os.makedirs(os.path.dirname(args.output), exist_ok=True)
        sim_data.to_csv(args.output, index=False)
        print(f"Simulated samples saved to: {args.output}")
    elif args.command == 'analyze':
        comprehensive_analysis(args.sim_csv, real_csv_path=args.real_csv, alpha=args.alpha, out_dir=args.out_dir)
    else:
        parser.print_help()


if __name__ == "__main__":
    main()

In [5]:
fit_copula_and_save('../data/processed/std_resids.csv', copula_type='clayton', out_path='models/best_copula.pkl')

Fitted clayton copula for FPT_DATA-HPG_DATA and saved to models/best_copula.pkl\clayton_FPT_DATA_HPG_DATA.pkl
Fitted clayton copula for FPT_DATA-MWG_DATA and saved to models/best_copula.pkl\clayton_FPT_DATA_MWG_DATA.pkl
Fitted clayton copula for FPT_DATA-VCB_DATA and saved to models/best_copula.pkl\clayton_FPT_DATA_VCB_DATA.pkl
Fitted clayton copula for FPT_DATA-VIC_DATA and saved to models/best_copula.pkl\clayton_FPT_DATA_VIC_DATA.pkl
Fitted clayton copula for HPG_DATA-MWG_DATA and saved to models/best_copula.pkl\clayton_HPG_DATA_MWG_DATA.pkl
Fitted clayton copula for HPG_DATA-VCB_DATA and saved to models/best_copula.pkl\clayton_HPG_DATA_VCB_DATA.pkl
Fitted clayton copula for HPG_DATA-VIC_DATA and saved to models/best_copula.pkl\clayton_HPG_DATA_VIC_DATA.pkl
Fitted clayton copula for MWG_DATA-VCB_DATA and saved to models/best_copula.pkl\clayton_MWG_DATA_VCB_DATA.pkl
Fitted clayton copula for MWG_DATA-VIC_DATA and saved to models/best_copula.pkl\clayton_MWG_DATA_VIC_DATA.pkl
Fitted cla

In [ ]:
best_copula_model, marginals = load_copula_and_marginals('models/best_copula.pkl', '../data/processed/std_resids.csv')
simulate_copula_portfolio(best_copula_model, marginals, n_sim=10000, random_state=None)